In [3]:
Full={}
Full_f={}
Full_f1={}
max_kin=2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
import matplotlib.pyplot as plt
import os,sys
import glob
import numpy as np
import tabulate
from tabulate import tabulate
from numpy import *
import scipy
from scipy import interpolate
from scipy.optimize import curve_fit

from scipy.interpolate import make_interp_spline, BSpline
from scipy.interpolate import griddata

sys.path.append('/home/jbane/headers')
import SQLFuncs as SQLF
import seaborn as sns


plt.style.use('seaborn-white')

plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 10
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 12


def Cross_table(tgt,table):
   
    #Fill the table once
    if len(table) != 10:
        for i in range(16):
            file_name = "/home/jbane/tritium/T2_externals/OUT/marathon_kin{}_{}.out".format(i,tgt)
            if os.path.isfile(file_name):
                table[i] = pd.read_table(file_name,delim_whitespace=True)
                
def interper(DFs,x,Q):
    f = interpolate.interp2d(DFs["x"],DFs["Q2"],DFs["Sig_Born"],kind='linear', 
                             copy=True,  bounds_error=True)
    sigma_born=f(x,Q)
    h = interpolate.interp2d(DFs["x"],DFs["Q2"],DFs["Sig_Rad"],kind='linear', 
                             copy=True, bounds_error=True )
    sigma_rad=h(x,Q)
    return sigma_born,sigma_rad  

def weighted_average(df,data_col,weight_col,by_col):
    df['_data_times_weight'] = df[data_col]*df[weight_col]
    df['_weight_where_notnull'] = df[weight_col]*pd.notnull(df[data_col])
    g = df.groupby(by_col)
    result = g['_data_times_weight'].sum() / g['_weight_where_notnull'].sum()
    del df['_data_times_weight'], df['_weight_where_notnull']
    return result

def wavg(group, avg_name, weight_name):
    """ http://stackoverflow.com/questions/10951341/pandas-dataframe-aggregate-
    function-using-multiple-columns. In rare instance, we may not have weights, 
    so just return the mean. Customize this if your business case
    should return otherwise.
    """
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()
    
def inquad(A,B):
    C=sqrt(pow(A,2)+pow(B,2))
    return C

def inquads(A):
    C=0
    for a in A:
        C=sqrt(pow(C,2)+pow(a,2))
    return C


markers =[[3,0,0] ,[5,1,90] ,[4,0,90],[3,1,90],[6,2,0] ,[5,0,0],[4,1,0],
          [7,0,0],[3,0,0] ,[5,1,90]]
colors=['red', 'Green', 'Blue','Maroon', 'Orange', 'Purple', 'Cyan', 
        'Magenta','Teal','Brown','Navy']

In [4]:
cuttype="tightcut/"

def all_tgt(tgt,bins):

    table={}
    Cross_table(tgt,table)
    
    Xaxis="Xbj" #theta / Xbj
    if Xaxis == "Xbj":
        xaxis="xbj"
    else :
        xaxis="theta"

   
        
    mcfile='/home/jbane/tritium/Tri_offline/MC_comparison/yield_output/{}{}bins/{}/'.format(cuttype,bins,xaxis)
    datafile='/home/jbane/tritium/Tri_offline/yield/yield_output/{}{}bins/{}/'.format(cuttype,bins,xaxis)
    kins={}
    kin={}
    sig_r={}
    sig_b={}
    sig_rr={}
    sig_bb={}

    kins=sorted(glob.glob('{}{}_kin?.dat'.format(mcfile,tgt)))
    kins+=sorted(glob.glob('{}{}_kin??.dat'.format(mcfile,tgt)))

    kin_num_array={}

    mcDF={}
    dataDF={}
    mcDFs={}
    dataDFs={}
    ratios={}
    i=0
    for i in range(len(kins)):
        kin[i] = kins[i].replace(mcfile,"")
        kin[i] = kin[i].replace('.dat',"")

        
        kin_str_num=kin[i].replace("{}_kin".format(tgt),"")
        kin_num=int(kin_str_num)
 
        if kin_num > max_kin :
            continue
        
        
        kin_str_num=kin[i].replace("{}_kin".format(tgt),"")
        kin_num=int(kin_str_num)   
        kin_num_array[i]=kin_num
        mcfileR=mcfile+'{}.dat'.format(kin[i])
        datafileR=datafile+'{}.dat'.format(kin[i])
        chk1=0
        chk=0
        if os.path.isfile(mcfileR):
            chk=1
        else:
            ("MC file does not exist") 
            chk=0
        if os.path.isfile(datafileR):
            chk1=1
        else :
            print("Data file does not exist")
            chk1=0
        if chk==0 or chk1==0:
            mcDF[i]=pd.DataFrame()
            dataDF[i]=pd.DataFrame()
            continue     

        mcDF[i]=pd.read_table(mcfileR,delim_whitespace=True)
        dataDF[i]=pd.read_table(datafileR,delim_whitespace=True)
        mcDF[i]['yield']=mcDF[i]['Yield']/1000.0
        mcDF[i]['Error']=mcDF[i]['Error']/1000.0

        dataDF[i]['yield']=dataDF[i]['Yield']
        mcmed=mcDF[i]['yield'].mean()
        datamed=dataDF[i]['yield'].mean()
        mcDFs[i]=mcDF[i][mcDF[i]['yield'] >=mcmed *0.5]
        dataDFs[i]=dataDF[i][dataDF[i]['yield'] >= datamed *0.5]
        mcDFs[i]=mcDFs[i].dropna()
        dataDFs[i]=dataDFs[i].dropna()
   
        if len(dataDFs[i]) <1 :
                continue
        #cut out the edge bins for now
        if bins > 24 :
            dataDFs[i]=dataDFs[i].drop(dataDFs[i].index[0])
            
        last= len(dataDFs[i].index)
        #dataDFs[i]=dataDFs[i].drop(dataDFs[i].index[last-1])
        if bins > 24 :
            mcDFs[i]=mcDFs[i].drop(mcDFs[i].index[0])
        ml=len(mcDFs[i].index)
        #mcDFs[i]=mcDFs[i].drop(mcDFs[i].index[ml-1])

        minind= dataDFs[i].index[0]
        maxind=dataDFs[i].index[len(dataDFs[i].index)-1]
        mcDFs[i]=mcDFs[i][mcDFs[i].index>=minind]
       # mcDFs[i]=mcDFs[i][mcDFs[i].index<=maxind]        
        if len(dataDFs[i]) >=5 :
            dataDFs[i]=dataDFs[i].drop(dataDFs[i].index[0])
            mcDFs[i]=mcDFs[i].drop(mcDFs[i].index[0])
        if len(dataDFs[i]) >=6 :
            dataDFs[i]=dataDFs[i].drop(dataDFs[i].index[0])
            last= len(dataDFs[i].index)
            dataDFs[i]=dataDFs[i].drop(dataDFs[i].index[last-1])
            mcDFs[i]=mcDFs[i].drop(mcDFs[i].index[0])
            ml=len(mcDFs[i].index)
            mcDFs[i]=mcDFs[i].drop(mcDFs[i].index[ml-1])
        if len(dataDFs[i]) >=7 :
            last= len(dataDFs[i].index)
            dataDFs[i]=dataDFs[i].drop(dataDFs[i].index[last-1])
            ml=len(mcDFs[i].index)
            mcDFs[i]=mcDFs[i].drop(mcDFs[i].index[ml-1])  
    #dataDFs[i]=dataDFs[i].drop(dataDFs[i].index[0])
    #dataDFs[i]=dataDFs[i].drop(dataDFs[i].index[last-2])
    

       # print("data",i, dataDFs[i])
       # print("MC  ",i,mcDFs[i])
        #dataDFs[i]= dataDFs[i].drop(dataDFs[i].index[[0,len(dataDFs[i])-1]])
        #mcDFs[i]=     mcDFs[i].drop(mcDFs[i].index[  [0,  len(mcDFs[i])-1]])
        dataDFs[i]['Stat_err']=1.0/np.sqrt((dataDFs[i]['Ne']))*dataDFs[i]['yield']
        ratios[i] = mcDFs[i].copy()
        ratios[i].rename(columns={'Ne':'MC_Ne'})
        ratios[i]['Data_Ne']=dataDFs[i]['Ne']
        ratios[i]['Data_xbjc']=dataDFs[i]['Xbjc']
        #ratios[i].drop('yield')
        ratios[i]["data_err"]=dataDFs[i]['Error']
        ratios[i]["data_err_stat"]=dataDFs[i]['Stat_err']
        ratios[i]['data_Y'] =dataDFs[i]['yield']
        ratios[i]['MC_Y'] = mcDFs[i]['yield']
        ratios[i]['ratio'] = dataDFs[i]['yield']/mcDFs[i]['yield']
        ratios[i]['offset'] = dataDFs[i]['yield'] - mcDFs[i]['yield']
        ratios[i]['ratio_err'] =ratios[i]['ratio']*np.sqrt(  pow((dataDFs[i]['Stat_err']/dataDFs[i]['yield']),2) + pow((mcDFs[i]['Error']/mcDFs[i]['yield']),2))
        
        ratios[i]['new_x'] = -ratios[i]['{}'.format(Xaxis)].mean()+ratios[i]['{}'.format(Xaxis)]    
            
            #pow((1/(np.sqrt(dataDFs[i]['Ne']))),2)  +   pow((1/(np.sqrt(mcDFs[i]['Ne']))),2)     )

        i=i+1

    import matplotlib._color_data as mcd
    overlap = [name for name in mcd.CSS4_COLORS if "xkcd:" + name in mcd.XKCD_COLORS]

    markers =[[3,0,0]  ,[5,1,90] ,[4,0,90],[3,1,90],[6,2,0] ,[5,0,0] ,[4,1,0],[7,0,0]]
    markers2 =[[3,0,0] ,[5,1,90] ,[4,0,90],[3,1,90],[6,2,0] ,[5,0,0],[4,1,0],[7,0,0]]
    colors=['red', 'Green', 'Blue', 'Orange', 'Purple', 'Cyan', 'Magenta','Teal','Brown','Maroon','Mint','Navy',]
    fig= plt.figure(dpi=500,figsize=(12,7))
    ax=fig.add_subplot(2,1,1)
    plt.title('{} Data to MC comparison'.format(tgt))
    bx=fig.add_subplot(2,1,2, sharex=ax)

    #fig,bx=plt.subplots(dpi=250)
    k=int(0)

    print("start second loop")
    for j in range(len(mcDF)):
        
        xkcd = mcd.XKCD_COLORS["xkcd:" + overlap[j]].upper()
        xkcd2 = mcd.XKCD_COLORS["xkcd:" + overlap[j+len(mcDF)]].upper()
        #print(j , kin[j])
        #print()
        if k == 7:
            k = 0
        if dataDF[j].empty or mcDF[j].empty:
            continue
        if(len(dataDFs[j])) == 0:
            continue
        if(len(mcDF[j])) ==0:
            continue



        kin_str_num=kin[j].replace("{}_kin".format(tgt),"")
        kin_num=int(kin_str_num)
        print(kin_num)
        if kin_num > max_kin :
            continue
        xbj=ratios[j]["Xbjc"]
        Q2=ratios[j]["Q2"]
     #   print( xbj,Q2)
        sig_bb[kin_num],sig_rr[kin_num] =interper(table[kin_num],xbj,Q2)
        print()
        #print(sig_bb[kin_num])
        #print()
        
        if len(sig_bb[kin_num]) > 1:
            sig_b[kin_num]=sig_bb[kin_num].diagonal()
        else:
            sig_b[kin_num]=sig_bb[kin_num]
        if len(sig_rr[kin_num]) > 1:
            sig_r[kin_num]=sig_rr[kin_num].diagonal()
        else:
            sig_r[kin_num]=sig_rr[kin_num]
            
            
        ratios[j]["Sigma_Born"]=sig_b[kin_num]
        ratios[j]["Sigma_Rad"]=sig_r[kin_num]
        ratios[j]["RCF"] =ratios[j]["Sigma_Born"]/ ratios[j]["Sigma_Rad"]
        ratios[j]["Sigma_Born_C"]=ratios[j]["Sigma_Born"]*ratios[j]["ratio"]
        ratios[j]["Sigma_Rad_C"]=ratios[j]["Sigma_Rad"]*ratios[j]["ratio"]
        ratios[j]["Sig_Born_err"]=ratios[j]["Sigma_Born_C"]* np.sqrt( pow( (ratios[j]['ratio_err']/ratios[j]["ratio"] ),2) )
        ratios[j]["Sig_Rad_err"]=ratios[j]["Sigma_Rad"]  * np.sqrt( pow( (ratios[j]['ratio_err']/ratios[j]["ratio"] ),2) )



            

     #   print(sig_b[kin_num])
     #   print(sig_r[kin_num])    
        #if j == 110:
         #   ax = mcDFs[j].plot(x='theta',y='yield',yerr=':error',kind='scatter', marker=markers[j], color='blue', label='MC run{}'.format(runs[j]))
        #else :
        Xaxis = "Xbjc"    
        xaxis = "Xbjc" 
    
        if j == 0:
        
            mcDFs[j].plot(ax=ax,x='{}'.format(Xaxis),y='yield',yerr='Error',kind='scatter',marker=markers[k], color='blue', label='Monte Carlo')
            dataDFs[j].plot(ax=ax,x='{}'.format(Xaxis),y='yield',yerr='Stat_err',kind='scatter', marker=markers2[k], color='red', label='Data ')
            #dataDFs[j].plot(ax=ax,x='{}'.format(Xaxis),y='yield',yerr='Error',kind='scatter', marker=markers2[k], color='red')
        else :
            mcDFs[j].plot(ax=ax,x='{}'.format(Xaxis),y='yield',yerr='Error',kind='scatter',marker=markers[k], color='blue')
            dataDFs[j].plot(ax=ax,x='{}'.format(Xaxis),y='yield',yerr='Error',kind='scatter', marker=markers2[k], color='red')
            
        ratios[j].plot(ax=bx,x='{}'.format(Xaxis) ,y='ratio',yerr='ratio_err', xerr='offset', kind='scatter', label='{}'.format(kin[j]),marker=markers2[k],c=colors[k] )
        k=k+1
        print(j , kin[j])    



        
      
    ax.set_yscale("log")
    ax.grid(True)
    bx.grid(True)
    
    
    ax.legend(ncol=2)

    handles, labels = ax.get_legend_handles_labels()
    print(handles,labels)
    
    #ax.get_legend().remove()
    
    if tgt == "H":
        bx.legend(ncol=2)
    else :
        bx.legend(ncol=4)    
    #bx.set_ylim(0.7,1.2)

    
   # fig.text(0.5, 0.5, 'Preliminary',
   #      fontsize=75, color='gray',
   #      ha='center', va='center', alpha=0.5)
    
    fig.subplots_adjust(hspace=0)
    fig.savefig('images/{}{}bins/{}_all.png'.format(cuttype,bins,tgt),bbox_inches='tight')
    fig.savefig('images/{}{}bins/{}_all.pdf'.format(cuttype,bins,tgt),bbox_inches='tight')
    fig.savefig('images/{}{}bins/{}_all.eps'.format(cuttype,bins,tgt),bbox_inches='tight')
    fig.savefig('/home/jbane/latex_shows/images/{}_all.pdf'.format(tgt),bbox_inches='tight')

  
    return ratios,kin,fig,dataDFs,mcDFs

def all_plots(Full,bins):
    D2=Full[0]
    He3=Full[1]
    H3=Full[2]

    fig3= plt.figure(dpi=400,figsize=(11,5))
    cx=fig3.add_subplot(1,1,1)
    plt.title('Coss Section by Monte Carlo Ratio')
    D2.plot(ax=cx,x="Xbj",y="Sigma_Born_C",yerr='Sig_Born_err',kind="scatter",label="D2")
    He3.plot(ax=cx,x="Xbj",y="Sigma_Born_C",yerr='Sig_Born_err',kind="scatter",color='r',marker='+',label="He3")
    H3.plot(ax=cx,x="Xbj",y="Sigma_Born_C",yerr='Sig_Born_err',kind="scatter",color='g',marker="x",label="H3")
    cx.set_yscale("log")
    plt.ylabel('$\sigma $ ')
    cx.grid(True)

    #fig1.subplots_adjust(hspace=0)
    fig3.savefig('images/{}{}bins/all_final.png'.format(cuttype,bins),bbox_inches='tight')
    return D2, He3, H3

def EMC(D2,He3,H3,bins):

    H3_s=H3[['Xbj','Sigma_Born_C','Sig_Born_err']]
    He3_s=He3[['Xbj','Sigma_Born_C','Sig_Born_err']]
    D2_s=D2[['Xbj','Sigma_Born_C','Sig_Born_err']]
    D2_s=D2_s.rename(columns={"Xbj":"D2_Xbj","Sigma_Born_C": "D2_Born", "Sig_Born_err": "D2_err"})

    EMC_H3  = pd.concat([D2_s,H3_s], axis=1)
    EMC_He3 = pd.concat([D2_s,He3_s],axis=1)

    EMC_H3 =EMC_H3.dropna()
    EMC_He3=EMC_He3.dropna()

    EMC_H3['EMC']  = ( EMC_H3['Sigma_Born_C']/ EMC_H3['D2_Born']) * (2/3) 
    EMC_H3['offset'] = abs(EMC_H3['Xbj'] - EMC_H3['D2_Xbj'])
    EMC_H3['EMC_err'] = EMC_H3['EMC'] * np.sqrt( pow( (EMC_H3['Sig_Born_err']/EMC_H3['Sigma_Born_C']),2) + pow( (EMC_H3['D2_err']/EMC_H3['D2_Born']),2) )

    EMC_He3['EMC'] = (EMC_He3['Sigma_Born_C']/EMC_He3['D2_Born']) * (2/3)
    EMC_He3['offset'] = abs(EMC_He3['Xbj'] - EMC_He3['D2_Xbj'])
    EMC_He3['EMC_err'] = EMC_He3['EMC'] * np.sqrt( pow( (EMC_He3['Sig_Born_err']/EMC_He3['Sigma_Born_C']),2) + pow( (EMC_He3['D2_err']/EMC_He3['D2_Born']),2) )

    fig4= plt.figure(dpi=500,figsize=(11,5))
    dx=fig4.add_subplot(1,1,1)
    plt.title('Cross section ratio ')

    EMC_He3.plot(ax=dx,x="Xbj",xerr='offset',y="EMC",yerr='EMC_err',kind="scatter",color='r',marker='+',label="He3/D")
    EMC_H3.plot( ax=dx,x="Xbj",xerr='offset',y="EMC",yerr='EMC_err',kind="scatter",color='g',marker="x",label="H3/D ")
    
    
    #Add in He3 datamined data
    He3df_un_mined = pd.read_csv("/home/jbane/Downloads/uncorrected.csv",names=['xbj','EMC'])
    He3df_un_mined.plot(ax=dx,x='xbj',y='EMC',color='b',marker='D',label="He3/D E3103")
    
    
    #dx.set_yscale("log")
    dx.set_ylim(0.75,1.2)
    plt.ylabel('($\sigma_A /3)  / (\sigma_D /2) $ ')
    dx.grid(True)

    #fig1.subplots_adjust(hspace=0)
    fig4.savefig('images/{}{}bins/EMC_final.png'.format(cuttype,bins),bbox_inches='tight')

    
def ISO_EMC(D2,He3,H3,bins):  
    H3_s=H3[['Xbj','Sigma_Born_C','Sig_Born_err']]
    He3_s=He3[['Xbj','Sigma_Born_C','Sig_Born_err']]
    D2_s=D2[['Xbj','Sigma_Born_C','Sig_Born_err']]
    D2_s=D2_s.rename(columns={"Xbj":"D2_Xbj","Sigma_Born_C": "D2_Born", "Sig_Born_err": "D2_err"})

    EMC_H3  = pd.concat([D2_s,H3_s], axis=1)
    EMC_He3 = pd.concat([D2_s,He3_s],axis=1)

    EMC_H3 =EMC_H3.dropna()
    EMC_He3=EMC_He3.dropna()

    EMC_H3['EMC']  = ( EMC_H3['Sigma_Born_C']/ EMC_H3['D2_Born']) * (2/3) 
    EMC_H3['offset'] = abs(EMC_H3['Xbj'] - EMC_H3['D2_Xbj'])
    EMC_H3['EMC_err'] = EMC_H3['EMC'] * np.sqrt( pow( (EMC_H3['Sig_Born_err']/EMC_H3['Sigma_Born_C']),2) + pow( (EMC_H3['D2_err']/EMC_H3['D2_Born']),2) )

    EMC_He3['EMC'] = (EMC_He3['Sigma_Born_C']/EMC_He3['D2_Born']) * (2/3)
    EMC_He3['offset'] = abs(EMC_He3['Xbj'] - EMC_He3['D2_Xbj'])
    EMC_He3['EMC_err'] = EMC_He3['EMC'] * np.sqrt( pow( (EMC_He3['Sig_Born_err']/EMC_He3['Sigma_Born_C']),2) + pow( (EMC_He3['D2_err']/EMC_He3['D2_Born']),2) )
    
    
    Fn_Fp= pd.read_csv("/home/jbane/Downloads/Fn_Fp.csv")
    F_n_p= interpolate.interp1d(Fn_Fp['xbj'],Fn_Fp['Fn/Fp'],kind='linear', copy=True, bounds_error=False ) 
    F_err= interpolate.interp1d(Fn_Fp['xbj'],Fn_Fp['Total_Error'],kind='linear', copy=True , bounds_error=False) 

        

    He3df_cor_mined = pd.read_csv("/home/jbane/Downloads/corrected.csv",names=['xbj','EMC'])
    He3df_uncor_mined = pd.read_csv("/home/jbane/Downloads/uncorrected.csv",names=['xbj','EMC'])
    
    fn_fp_r = He3df_cor_mined
    fn_fp_r['Fn/Fp']=  He3df_uncor_mined['EMC']/ He3df_cor_mined['EMC']
    fun_fn_fp = interpolate.interp1d(Fn_Fp['xbj'],Fn_Fp['Fn/Fp'],kind='linear', copy=True , bounds_error=False)
    

    
    EMC_H3['Iso_fac'] = (0.5*(1.0 + F_n_p(EMC_H3['Xbj'])))/((1/3.0)*(1+2*F_n_p(EMC_H3['Xbj'])))
    EMC_He3['Iso_fac'] = (0.5*(1.0 + F_n_p(EMC_He3['Xbj'])))/((1/3.0)*(2+1*F_n_p(EMC_He3['Xbj'])))
    EMC_H3['Iso_fac_E3'] = (0.5*(1.0 + fun_fn_fp(EMC_H3['Xbj'])))/((1/3.0)*(1+2*fun_fn_fp(EMC_H3['Xbj'])))
    EMC_He3['Iso_fac_E3'] = (0.5*(1.0 + fun_fn_fp(EMC_He3['Xbj'])))/((1/3.0)*(2+1*fun_fn_fp(EMC_He3['Xbj'])))
    
    EMC_H3['Iso_EMC'] =EMC_H3['EMC']*EMC_H3['Iso_fac']
    EMC_He3['Iso_EMC'] =EMC_He3['EMC']*EMC_He3['Iso_fac']
    
    EMC_H3['Iso_EMC_E3'] =EMC_H3['EMC']*EMC_H3['Iso_fac_E3']
    EMC_He3['Iso_EMC_E3'] =EMC_He3['EMC']*EMC_He3['Iso_fac_E3']
    
    #Add in He3 datamined data
    #He3df_un_mined = pd.read_csv("/home/jbane/Downloads/uncorrected.csv",names=['xbj','EMC'])
    #He3df_un_mined.plot(ax=dx,x='xbj',y='EMC',color='b',marker='D',label="E3103_uncorrected")
    fig5= plt.figure(dpi=500,figsize=(11,5))
    hx=fig5.add_subplot(1,1,1)
    plt.title('Cross section ratio Isoscalar corrected')
    
    
    #E3103
    EMC_He3.plot(ax=hx,x="Xbj",xerr='offset',y="Iso_EMC",yerr='EMC_err',kind="scatter",color='r',marker='+',label="He3/D")
    EMC_H3.plot( ax=hx,x="Xbj",xerr='offset',y="Iso_EMC",yerr='EMC_err',kind="scatter",color='g',marker="x",label="H3/D ")
    EMC_He3.plot(ax=hx,x="Xbj",xerr='offset',y="Iso_EMC_E3",yerr='EMC_err',kind="scatter",color='r',marker='P',label="He3/D E3")
    EMC_H3.plot( ax=hx,x="Xbj",xerr='offset',y="Iso_EMC_E3",yerr='EMC_err',kind="scatter",color='g',marker="p",label="H3/D E3")
    
    He3df_cor_mined.plot(ax=hx,x='xbj',y='EMC',color='c',marker='H',label="He3 E3103")

    hx.set_ylim(0.75,1.2)
    plt.ylabel('($\sigma_A /3)  / (\sigma_D /2) $ ')
    hx.grid(True)

    
def EMC(D2,He3,H3,bins):

    H3_s=H3[['Xbj','Sigma_Born_C','Sig_Born_err']]
    He3_s=He3[['Xbj','Sigma_Born_C','Sig_Born_err']]
    D2_s=D2[['Xbj','Sigma_Born_C','Sig_Born_err']]
    D2_s=D2_s.rename(columns={"Xbj":"D2_Xbj","Sigma_Born_C": "D2_Born", "Sig_Born_err": "D2_err"})

    EMC_H3  = pd.concat([D2_s,H3_s], axis=1)
    EMC_He3 = pd.concat([D2_s,He3_s],axis=1)

    EMC_H3 =EMC_H3.dropna()
    EMC_He3=EMC_He3.dropna()

    EMC_H3['EMC']  = ( EMC_H3['Sigma_Born_C']/ EMC_H3['D2_Born']) * (2/3) 
    EMC_H3['offset'] = abs(EMC_H3['Xbj'] - EMC_H3['D2_Xbj'])
    EMC_H3['EMC_err'] = EMC_H3['EMC'] * np.sqrt( pow( (EMC_H3['Sig_Born_err']/EMC_H3['Sigma_Born_C']),2) + pow( (EMC_H3['D2_err']/EMC_H3['D2_Born']),2) )

    EMC_He3['EMC'] = (EMC_He3['Sigma_Born_C']/EMC_He3['D2_Born']) * (2/3)
    EMC_He3['offset'] = abs(EMC_He3['Xbj'] - EMC_He3['D2_Xbj'])
    EMC_He3['EMC_err'] = EMC_He3['EMC'] * np.sqrt( pow( (EMC_He3['Sig_Born_err']/EMC_He3['Sigma_Born_C']),2) + pow( (EMC_He3['D2_err']/EMC_He3['D2_Born']),2) )

    fig4= plt.figure(dpi=500,figsize=(11,5))
    dx=fig4.add_subplot(1,1,1)
    plt.title('Cross section ratio ')

    EMC_He3.plot(ax=dx,x="Xbj",xerr='offset',y="EMC",yerr='EMC_err',kind="scatter",color='r',marker='+',label="He3/D")
    EMC_H3.plot( ax=dx,x="Xbj",xerr='offset',y="EMC",yerr='EMC_err',kind="scatter",color='g',marker="x",label="H3/D ")
    
    
    #Add in He3 datamined data
    He3df_un_mined = pd.read_csv("/home/jbane/Downloads/uncorrected.csv",names=['xbj','EMC'])
    He3df_un_mined.plot(ax=dx,x='xbj',y='EMC',color='b',marker='D',label="He3/D E3103")
    
    
    #dx.set_yscale("log")
    dx.set_ylim(0.75,1.2)
    plt.ylabel('($\sigma_A /3)  / (\sigma_D /2) $ ')
    dx.grid(True)

    #fig1.subplots_adjust(hspace=0)
    fig4.savefig('images/{}{}bins/EMC_final.png'.format(cuttype,bins),bbox_inches='tight')

    
def ISO_EMC(D2,He3,H3,bins):  
    H3_s=H3[['Xbj','Sigma_Born_C','Sig_Born_err']]
    He3_s=He3[['Xbj','Sigma_Born_C','Sig_Born_err']]
    D2_s=D2[['Xbj','Sigma_Born_C','Sig_Born_err']]
    D2_s=D2_s.rename(columns={"Xbj":"D2_Xbj","Sigma_Born_C": "D2_Born", "Sig_Born_err": "D2_err"})

    EMC_H3  = pd.concat([D2_s,H3_s], axis=1)
    EMC_He3 = pd.concat([D2_s,He3_s],axis=1)

    EMC_H3 =EMC_H3.dropna()
    EMC_He3=EMC_He3.dropna()

    EMC_H3['EMC']  = ( EMC_H3['Sigma_Born_C']/ EMC_H3['D2_Born']) * (2/3) 
    EMC_H3['offset'] = abs(EMC_H3['Xbj'] - EMC_H3['D2_Xbj'])
    EMC_H3['EMC_err'] = EMC_H3['EMC'] * np.sqrt( pow( (EMC_H3['Sig_Born_err']/EMC_H3['Sigma_Born_C']),2) + pow( (EMC_H3['D2_err']/EMC_H3['D2_Born']),2) )

    EMC_He3['EMC'] = (EMC_He3['Sigma_Born_C']/EMC_He3['D2_Born']) * (2/3)
    EMC_He3['offset'] = abs(EMC_He3['Xbj'] - EMC_He3['D2_Xbj'])
    EMC_He3['EMC_err'] = EMC_He3['EMC'] * np.sqrt( pow( (EMC_He3['Sig_Born_err']/EMC_He3['Sigma_Born_C']),2) + pow( (EMC_He3['D2_err']/EMC_He3['D2_Born']),2) )
    
    
    Fn_Fp= pd.read_csv("/home/jbane/Downloads/Fn_Fp.csv")
    F_n_p= interpolate.interp1d(Fn_Fp['xbj'],Fn_Fp['Fn/Fp'],kind='linear', copy=True, bounds_error=False ) 
    F_err= interpolate.interp1d(Fn_Fp['xbj'],Fn_Fp['Total_Error'],kind='linear', copy=True , bounds_error=False) 

        

    He3df_cor_mined = pd.read_csv("/home/jbane/Downloads/corrected.csv",names=['xbj','EMC'])
    He3df_uncor_mined = pd.read_csv("/home/jbane/Downloads/uncorrected.csv",names=['xbj','EMC'])
    
    fn_fp_r = He3df_cor_mined
    fn_fp_r['Fn/Fp']=  He3df_uncor_mined['EMC']/ He3df_cor_mined['EMC']
    fun_fn_fp = interpolate.interp1d(Fn_Fp['xbj'],Fn_Fp['Fn/Fp'],kind='linear', copy=True , bounds_error=False)
    

    
    EMC_H3['Iso_fac'] = (0.5*(1.0 + F_n_p(EMC_H3['Xbj'])))/((1/3.0)*(1+2*F_n_p(EMC_H3['Xbj'])))
    EMC_He3['Iso_fac'] = (0.5*(1.0 + F_n_p(EMC_He3['Xbj'])))/((1/3.0)*(2+1*F_n_p(EMC_He3['Xbj'])))
    EMC_H3['Iso_fac_E3'] = (0.5*(1.0 + fun_fn_fp(EMC_H3['Xbj'])))/((1/3.0)*(1+2*fun_fn_fp(EMC_H3['Xbj'])))
    EMC_He3['Iso_fac_E3'] = (0.5*(1.0 + fun_fn_fp(EMC_He3['Xbj'])))/((1/3.0)*(2+1*fun_fn_fp(EMC_He3['Xbj'])))
    
    EMC_H3['Iso_EMC'] =EMC_H3['EMC']*EMC_H3['Iso_fac']
    EMC_He3['Iso_EMC'] =EMC_He3['EMC']*EMC_He3['Iso_fac']
    
    EMC_H3['Iso_EMC_E3'] =EMC_H3['EMC']*EMC_H3['Iso_fac_E3']
    EMC_He3['Iso_EMC_E3'] =EMC_He3['EMC']*EMC_He3['Iso_fac_E3']
    
    #Add in He3 datamined data
    #He3df_un_mined = pd.read_csv("/home/jbane/Downloads/uncorrected.csv",names=['xbj','EMC'])
    #He3df_un_mined.plot(ax=dx,x='xbj',y='EMC',color='b',marker='D',label="E3103_uncorrected")
    fig5= plt.figure(dpi=500,figsize=(11,5))
    hx=fig5.add_subplot(1,1,1)
    plt.title('Cross section ratio Isoscalar corrected')
    
    
    #E3103
    EMC_He3.plot(ax=hx,x="Xbj",xerr='offset',y="Iso_EMC",yerr='EMC_err',kind="scatter",color='r',marker='+',label="He3/D")
    EMC_H3.plot( ax=hx,x="Xbj",xerr='offset',y="Iso_EMC",yerr='EMC_err',kind="scatter",color='g',marker="x",label="H3/D ")
    EMC_He3.plot(ax=hx,x="Xbj",xerr='offset',y="Iso_EMC_E3",yerr='EMC_err',kind="scatter",color='r',marker='P',label="He3/D E3")
    EMC_H3.plot( ax=hx,x="Xbj",xerr='offset',y="Iso_EMC_E3",yerr='EMC_err',kind="scatter",color='g',marker="p",label="H3/D E3")
    
    He3df_cor_mined.plot(ax=hx,x='xbj',y='EMC',color='c',marker='H',label="He3 E3103")

    hx.set_ylim(0.75,1.2)
    plt.ylabel('($\sigma_A /3)  / (\sigma_D /2) $ ')
    hx.grid(True)
    
    
def ISO_EMC_both(D2,He3,H3,bins):  
    H3_s=H3[['Xbj','Sigma_Born_C','Sig_Born_err']]
    He3_s=He3[['Xbj','Sigma_Born_C','Sig_Born_err']]
    D2_s=D2[['Xbj','Sigma_Born_C','Sig_Born_err']]
    D2_s=D2_s.rename(columns={"Xbj":"D2_Xbj","Sigma_Born_C": "D2_Born", "Sig_Born_err": "D2_err"})

    EMC_H3  = pd.concat([D2_s,H3_s], axis=1)
    EMC_He3 = pd.concat([D2_s,He3_s],axis=1)

    EMC_H3 =EMC_H3.dropna()
    EMC_He3=EMC_He3.dropna()

    EMC_H3['EMC']  = ( EMC_H3['Sigma_Born_C']/ EMC_H3['D2_Born']) * (2/3) 
    EMC_H3['offset'] = abs(EMC_H3['Xbj'] - EMC_H3['D2_Xbj'])
    EMC_H3['EMC_err'] = EMC_H3['EMC'] * np.sqrt( pow( (EMC_H3['Sig_Born_err']/EMC_H3['Sigma_Born_C']),2) + pow( (EMC_H3['D2_err']/EMC_H3['D2_Born']),2) )

    EMC_He3['EMC'] = (EMC_He3['Sigma_Born_C']/EMC_He3['D2_Born']) * (2/3)
    EMC_He3['offset'] = abs(EMC_He3['Xbj'] - EMC_He3['D2_Xbj'])
    EMC_He3['EMC_err'] = EMC_He3['EMC'] * np.sqrt( pow( (EMC_He3['Sig_Born_err']/EMC_He3['Sigma_Born_C']),2) + pow( (EMC_He3['D2_err']/EMC_He3['D2_Born']),2) )
    
    
    Fn_Fp= pd.read_csv("/home/jbane/Downloads/Fn_Fp.csv")
    F_n_p= interpolate.interp1d(Fn_Fp['xbj'],Fn_Fp['Fn/Fp'],kind='linear', copy=True, bounds_error=False ) 
    F_err= interpolate.interp1d(Fn_Fp['xbj'],Fn_Fp['Total_Error'],kind='linear', copy=True , bounds_error=False) 

        

    He3df_cor_mined = pd.read_csv("/home/jbane/Downloads/corrected.csv",names=['xbj','EMC'])
    He3df_uncor_mined = pd.read_csv("/home/jbane/Downloads/uncorrected.csv",names=['xbj','EMC'])
    
    fn_fp_r = He3df_cor_mined
    fn_fp_r['Fn/Fp']=  He3df_uncor_mined['EMC']/ He3df_cor_mined['EMC']
    fun_fn_fp = interpolate.interp1d(Fn_Fp['xbj'],Fn_Fp['Fn/Fp'],kind='linear', copy=True , bounds_error=False)
    
    #p=[0.9377,-1.224,0.7604]
    p=[1.019,-1.317,0.8194]
    
    function_fn_fp= lambda x: x**2*p[2] + x*p[1] + p[0]
       
    EMC_H3['Iso_fac'] = (0.5*(1.0 + F_n_p(EMC_H3['Xbj'])))/((1/3.0)*(1+2*F_n_p(EMC_H3['Xbj'])))
    EMC_He3['Iso_fac'] = (0.5*(1.0 + F_n_p(EMC_He3['Xbj'])))/((1/3.0)*(2+1*F_n_p(EMC_He3['Xbj'])))
    EMC_H3['Iso_fac_E3'] = (0.5*(1.0 + fun_fn_fp(EMC_H3['Xbj'])))/((1/3.0)*(1+2*fun_fn_fp(EMC_H3['Xbj'])))
    EMC_He3['Iso_fac_E3'] = (0.5*(1.0 + fun_fn_fp(EMC_He3['Xbj'])))/((1/3.0)*(2+1*fun_fn_fp(EMC_He3['Xbj'])))

    EMC_H3['Iso_fac_tong'] = (0.5*(1.0 + function_fn_fp(EMC_H3['Xbj'])))/((1/3.0)*(1+2*function_fn_fp(EMC_H3['Xbj'])))
    EMC_He3['Iso_fac_tong'] = (0.5*(1.0 + function_fn_fp(EMC_He3['Xbj'])))/((1/3.0)*(2+1*function_fn_fp(EMC_He3['Xbj'])))
   
    EMC_H3['Iso_EMC'] =EMC_H3['EMC']*EMC_H3['Iso_fac']
    EMC_He3['Iso_EMC'] =EMC_He3['EMC']*EMC_He3['Iso_fac']

    EMC_H3['Iso_EMC_E3'] =EMC_H3['EMC']*EMC_H3['Iso_fac_E3']
    EMC_He3['Iso_EMC_E3'] =EMC_He3['EMC']*EMC_He3['Iso_fac_E3']

    EMC_H3['Iso_EMC_tong'] =EMC_H3['EMC']*EMC_H3['Iso_fac_tong']
    EMC_He3['Iso_EMC_tong'] =EMC_He3['EMC']*EMC_He3['Iso_fac_tong']

    
    #Add in He3 datamined data
    #He3df_un_mined = pd.read_csv("/home/jbane/Downloads/uncorrected.csv",names=['xbj','EMC'])
    #He3df_un_mined.plot(ax=dx,x='xbj',y='EMC',color='b',marker='D',label="E3103_uncorrected")
    fig5= plt.figure(dpi=500,figsize=(11,9))
    fx=plt.subplot(2,1,1)
    fx.set(ylabel='($\sigma_A /3)  / (\sigma_D /2) $ ')
    #plt.ylabel('($\sigma_A /3)  / (\sigma_D /2) $ ')
    
    plt.title('Cross section ratio Isoscalar corrected')
    
    gx=plt.subplot(2,1,2, sharex=fx, sharey=fx) 
   #gx.set(ylabel='($\sigma_A /3)  / (\sigma_D /2) $ ')
    #E3103

    #EMC_He3.plot(ax=fx,x="Xbj",xerr='offset',y="Iso_EMC_E3",yerr='EMC_err',kind="scatter",color='m',marker='*',label="He3/D Iso E3")
    EMC_He3.plot(ax=fx,x="Xbj",xerr='offset',y="Iso_EMC_tong",yerr='EMC_err',kind="scatter",color='k',marker='D',label="He3/D Iso Tong")
    EMC_He3.plot(ax=fx,x="Xbj",xerr='offset',y="EMC",yerr='EMC_err',kind="scatter",color='g',marker='p',label="He3/D")
    EMC_He3.plot(ax=fx,x="Xbj",xerr='offset',y="Iso_EMC",yerr='EMC_err',kind="scatter",color='r',marker='+',label="He3/D Iso")

    EMC_H3.plot( ax=gx,x="Xbj",xerr='offset',y="Iso_EMC",yerr='EMC_err',kind="scatter",color='g',marker="x",label="H3/D Iso")
    EMC_H3.plot( ax=gx,x="Xbj",xerr='offset',y="Iso_EMC_tong",yerr='EMC_err',kind="scatter",color='k',marker="D",label="H3/D Iso Tong")
    EMC_H3.plot( ax=gx,x="Xbj",xerr='offset',y="EMC",yerr='EMC_err',kind="scatter",color='b',marker="p",label="H3/D ")
    
    He3df_cor_mined.plot(ax=fx,x='xbj',y='EMC',color='m',marker='h',label="He3 E3103 cor")
    He3df_uncor_mined.plot(ax=fx,x='xbj',y='EMC',color='c',marker='h',label="He3 E3103 uncor")
    fx.set_ylim(0.75,1.2)
    fx.set_ylabel('($\sigma_A /3)  / (\sigma_D /2) $ ')
    fx.grid(True)    

    gx.set_ylim(0.7,1.1)
    plt.ylabel('($\sigma_A /3)  / (\sigma_D /2) $ ')
    gx.grid(True)    
    fig5.subplots_adjust(hspace=0)
    fig5.savefig('images/{}{}bins/EMC_iso_final.png'.format(cuttype,bins),bbox_inches='tight')
    
    
    

def ISO_Cor_EMC(EMC_A,tgt_name,A_num,Z_num,bins):
    
    #fig= plt.figure(dpi=500,figsize=(11,9))
    #plt.title('Cross section ratio Isoscalar corrected')
    #x=fig.add_subplot(1,1,1)
    
    
    Fn_Fp= pd.read_csv("/home/jbane/Downloads/Fn_Fp.csv")
    F_n_p= interpolate.interp1d(Fn_Fp['xbj'],Fn_Fp['Fn/Fp'],kind='linear', copy=True, bounds_error=False ) 
    F_err= interpolate.interp1d(Fn_Fp['xbj'],Fn_Fp['Total_Error'],kind='linear', copy=True , bounds_error=False)

    p=[1.019,-1.317,0.8194]
    
    function_fn_fp= lambda x: x**2*p[2] + x*p[1] + p[0]
    
    EMC_A['Iso_fac'] = (0.5*(1.0 + F_n_p(EMC_A['Xbj'])))/((1/A_num)*(Z_num+(A_num-Z_num)*F_n_p(EMC_A['Xbj'])))
    EMC_A['Iso_EMC'] =EMC_A['EMC']*EMC_A['Iso_fac']
    
    ##ADD in aditional error from correction
    #EMC_A.plot( ax=x,x="Xbj",xerr='offset',y="Iso_EMC",yerr='EMC_err',kind="scatter",color='g',marker="x",label="{}/D Iso".format(tgt_name))
    #x.set_ylim(0.75,1.2)
    #x.grid(True)
    return EMC_A
    
def kincenteral(ratios,kins):
    
    tgt=kins[0].replace("_kin1","")
    markers =[[3,0,0] ,[5,1,90] ,[4,0,90],[3,1,90],[6,2,0] ,[5,0,0],[4,1,0],[7,0,0],[3,0,0] ,[5,1,90] ,[4,0,90],[3,1,90],[6,2,0] ,[5,0,0],[4,1,0],[7,0,0]]
    colors=['red', 'Green', 'Blue', 'Orange', 'Purple', 'Cyan', 'Magenta','Teal','Brown','Maroon','Mint','Navy',]
    fig= plt.figure(dpi=100)
    xf=fig.add_subplot(1,1,1)   
    
    
    a=[]
    for i in range( len(ratios)) :
        a.append(ratios[i][['ratio','new_x','ratio_err']])
    b=pd.concat(a)
    
    
    from sklearn import datasets, linear_model
    x=b['new_x']
    y=b['ratio']
    yerr=b['ratio_err']

    length=len(x)
    x = x.values.reshape(length, 1)
    y = y.values.reshape(length, 1)
    yerr = yerr.values.reshape(length, 1)

    regr = linear_model.LinearRegression()
    #regr.fit(x, y)

    
    
    for i in range(len(ratios)):
        mean = ratios[i]["Xbj"].mean()
        ratios[i]["new_x"] = -(mean - ratios[i]["Xbj"])
        ratios[i].plot(ax=xf, x='new_x',y='ratio',yerr='ratio_err',kind='scatter',marker=markers[i], color=colors[i], label='kin {}'.format(kins[i]))
     
   # plt.plot(x, regr.predict(x),color='red', linewidth=3)
    
    xf.legend(ncol=2)
    xf.set_xlabel("Xbj - center Xbj")
    xf.grid(True)
        
    fig.savefig("/home/jbane/images/{}_centerkin.eps".format(tgt),bbox_inches='tight')
    return regr

def together(tgt,ratios,bins):
    
    if tgt =="D2":
        sel=0
    elif tgt =="He3":
        sel=1
    elif tgt =="H3":
        sel=2
    elif tgt =="C12":
        sel=3
    elif tgt =="H":
        sel=4
        
    Full_f[sel] = pd.concat(ratios)
   ## print(type(ratio_a) , type(ratio_a[1]))
    
    
    
    #Full[sel]=weighted_average(Full_f[sel],
    
    #Full[sel]=Full_f[sel].groupby(level=0).apply(wavg, Full_f[sel].columns, 'Sig_Born_err')

    cname =Full_f[sel].columns
    numofc=len(cname)
    ab={}

    #ac = wavg(aa, cname,"Sig_Born_err")
    for i,col in enumerate(cname):
        ab[i]=Full_f[sel].groupby(level=1).apply(wavg,col ,"Ne")

    Full[sel]= pd.DataFrame.from_dict(ab)
    Full[sel].columns = cname
       
    
    fig1= plt.figure(dpi=500,figsize=(9,5))
    cx=fig1.add_subplot(1,1,1)
    plt.title('{} Coss Section by Monte Carlo Ratio'.format(tgt))
    Full[sel].plot(ax=cx,x="Xbj",y="Sigma_Born_C",yerr='Sig_Born_err',c='r',kind="scatter",label="data")
    Full[sel].plot(ax=cx,x="Xbj",y="Sigma_Born",marker="D",c='b',kind="scatter", label="model",s=10)
    
    cx.set_yscale("log")
    plt.ylabel('$\sigma $ ')
    cx.grid(True)
   # fig1.text(0.5, 0.5, 'Preliminary',
    #     fontsize=75, color='gray',
     #    ha='center', va='center', alpha=0.5)
    #fig1.subplots_adjust(hspace=0)
    fig1.savefig('images/{}{}bins/{}_final.png'.format(cuttype,bins,tgt),bbox_inches='tight')
    fig1.savefig('images/{}{}bins/{}_final.pdf'.format(cuttype,bins,tgt),bbox_inches='tight')
    fig1.savefig('images/{}{}bins/{}_final.eps'.format(cuttype,bins,tgt),bbox_inches='tight')
    fig1.savefig('/home/jbane/latex_shows/images/{}_final.pdf'.format(tgt),bbox_inches='tight')
    return Full[sel]

def cs_dmc_r(Full,tgt):

    newfig = plt.figure(dpi=500,figsize=(11,5))
    axis = newfig.add_subplot(2,1,1)
    plt.title('{} Coss Section by Monte Carlo Ratio'.format(tgt))
    Full.plot(ax=axis,x="Xbj",y="Sigma_Born_C",yerr='Sig_Born_err',c='r',kind="scatter",label="data")
    Full.plot(ax=axis,x="Xbj",y="Sigma_Born",marker="D",c='b',kind="scatter", label="model",s=10)

    newdf=Full[['Xbj','Sigma_Born','Sigma_Born_C','Sig_Born_err']].copy()
    newdf['ratio']=newdf['Sigma_Born_C']/newdf['Sigma_Born']
    newdf['ratio_err'] = newdf['ratio'] * np.sqrt( (newdf['Sig_Born_err']/newdf['Sigma_Born_C'])**2 )
    axis2 = newfig.add_subplot(2,1,2,sharex=axis)
    newdf.plot(ax=axis2,y="ratio",x='Xbj',yerr='ratio_err',kind="scatter",color='r',marker='*',label="Sigma ratio" )

    axis.set_ylabel('Sigma ')
    axis.set_yscale("log")
    newfig.subplots_adjust(hspace=0)
    axis.grid(True)
    axis2.grid(True)
    newfig.savefig('images/{}_ratio.eps'.format(tgt),bbox_inches='tight')

def EMC_one(A,tgt_name,A_num,D2,bins,comp=""):

    A_s=A[['Xbjc','Sigma_Born_C','Sig_Born_err',"Sigma_Born"]]
    A_s.drop(A_s[A_s.Sigma_Born_C<=0.0000001].index)
    
    D2_s=D2[['Xbjc','Sigma_Born_C','Sig_Born_err',"Sigma_Born"]]
    D2_s=D2_s.rename(columns={"Xbjc":"D2_Xbj","Sigma_Born_C": "D2_Born", "Sig_Born_err": "D2_err","Sigma_Born": "Sigma_Born_D2"})

    EMC_A  = pd.concat([D2_s,A_s], axis=1)
    
    
    EMC_A =EMC_A.dropna()
    
    EMC_A['EMC']  = ( EMC_A['Sigma_Born_C']/ EMC_A['D2_Born']) * (2/(A_num*1.0)) 
    EMC_A['offset'] = abs(EMC_A['Xbjc'] - EMC_A['D2_Xbj'])
    EMC_A['EMC_err'] = EMC_A['EMC'] * np.sqrt( pow( (EMC_A['Sig_Born_err']/EMC_A['Sigma_Born_C']),2) + pow( (EMC_A['D2_err']/EMC_A['D2_Born']),2) )
    EMC_A =EMC_A.dropna() 
    fig= plt.figure(dpi=500,figsize=(11,5))
    x=fig.add_subplot(1,1,1)
    plt.title('Cross section ratio ')

    EMC_A.plot( ax=x,x="Xbjc",xerr='offset',y="EMC",yerr='EMC_err',kind="scatter",color='g',marker="x",label="{}/D $\sigma$ ratio from MARATHON ".format(tgt_name))
    if comp.find('3')  > -1:
        file="/home/jbane/emc_data/{}_maraEMC.dat".format(tgt_name,dtype=float)
        MARemc=pd.read_csv(file)
        MARemc.plot(ax=x,x='x',y='ratio',yerr='error',kind='scatter',marker='o',color='navy',label="MARATHON yield ratio method")
    
    if comp.find('1') > -1:
        EMC_A['X_ext']=EMC_A['Xbjc'] #+ 0.01
        EMC_A['ext_EMC'] = ( EMC_A['Sigma_Born']/ EMC_A['Sigma_Born_D2']) * (2/(A_num*1.0))
        EMC_A['ext_Err']=EMC_A['ext_EMC'] * 0.01
        model=""
        if tgt_name=="C12" :
            model = "Bodek F$_{2d}$ + SLAC EMC + NMC1992 $ F_{2n} / F_{2p}$  "
        else:
            model = "Bodek F$_{2d}$ + K/P EMC"
        #EMC_A.plot( ax=x,x="X_ext",y="ext_EMC",yerr='ext_Err',kind="scatter",color='b',marker="d",label=model)
        x.fill_between(x=EMC_A["X_ext"],y1=EMC_A["ext_EMC"]+EMC_A['ext_Err'],y2=EMC_A["ext_EMC"]-EMC_A['ext_Err'],color='b',label=model)
    
    if comp.find('2')  > -1:
        E03103df={}
        E03color=['lightsalmon','salmon','crimson','coral','orangered']
        files = glob.glob('/home/jbane/emc_data/{}_q2*.dat'.format(tgt_name))
        
        for i in range(len(files)):
            file=files[i]
            Q2=file[file.find('q2')+3:file.find('q2')+6]
            header =['x','xi','Q2','W2','Ratio','Stat','Sys','Iso. corr.','Coul. corr.']
            exclude = [i for i, line in enumerate(open(file)) if not line.startswith(' 0') or line.startswith('** ') ]
            E03103df[i] = pd.read_table(file,delim_whitespace=True,header=None,names=header,skiprows = exclude[0:],dtype=float)
            E03103df[i]=E03103df[i].drop(index=0).copy()
            E03103df[i]['Ratio_n_iso'] =E03103df[i]['Ratio']/E03103df[i]['Iso. corr.']
            E03103df[i]['err'] = E03103df[i]['Ratio']*np.sqrt( pow(E03103df[i]['Stat'],2) + pow(E03103df[i]['Sys'],2))
            E03103df[i].plot(ax=x,x='x',y='Ratio_n_iso',yerr='err',kind='scatter',marker='1',c=E03color[i],label="E03103 Q2={}".format(Q2))
            #x.fill_between(x=E03103df[i]["x"],y1=E03103df[i]["Ratio"]+E03103df[i]['err'],y2=E03103df[i]["Ratio"]-E03103df[i]['err'],color=E03color[i],label="E03103 Q2={}".format(Q2))
        
        
       # plt.fill_between("x_ext",'ext_Err','ext_Err',data=EMC_A)
    
    #Add in He3 datamined data
    #He3df_un_mined = pd.read_csv("/home/jbane/Downloads/uncorrected.csv",names=['xbj','EMC'])
    #He3df_un_mined.plot(ax=dx,x='xbj',y='EMC',color='b',marker='D',label="He3/D E3103")
    
    
    #dx.set_yscale("log")
    #x.set_ylim(0.8,1.2)    
    plt.ylabel('($\sigma_A /{})  / (\sigma_D /2) $ '.format(A_num))
    x.grid(True)
    plt.xlabel('$ X_{BJ} $')
    x.legend(ncol=2)
    #fig1.subplots_adjust(hspace=0)
    fig.savefig('images/{}{}bins/EMC_{}.png'.format(cuttype,bins,tgt_name),bbox_inches='tight')
    fig.savefig('images/{}{}bins/EMC_{}.eps'.format(cuttype,bins,tgt_name),bbox_inches='tight')
    return EMC_A
